In [1]:
import duckdb

In [2]:
conn = duckdb.connect('../fetch.db')

# Data Quality findings from part 2 process
### Receipts findings
+ Each receipt item does not have an individual pointsEarned value
+ RewardsReceiptItemList is will not always be the same size as purchasedItemCount
+ Many columns in the receiptitemslist have the same data values across the whole row

### Brands findings
+ Some barcodes are repeated
+ Some brand codes are repeated

# Data Quality in 'What are the top 5 brands by receipts scanned for most recent month?'
+ What does this mean?
    + Top 5 by number of receipts scanned with a specific brand code
    + Top 5 by number of receipt items scanned with a specific brand code

### ISSUE 1: Are we really trying to understand the most recent month?
+ The max date_scanned is from March 1st
+ Looking at counts of _receipt_ids by month scanned, it seems this exercise was meant to compare February 2021 to January 2021

In [3]:
conn.sql("""
         SELECT MAX(date_scanned) FROM receipts
         """).show()
print("\nUnique Receipts Scanned by Month")
conn.sql("""
         SELECT DATE_TRUNC('month', date_scanned) as month_scanned, COUNT(DISTINCT _receipt_id) FROM receipts
         GROUP BY month_scanned
         ORDER BY month_scanned DESC
         """).show()
print("\nUnique Receipts and Receipt Items Scanned by Month (Only receipts with receipt_items included)")
conn.sql("""
         WITH months_scanned AS (
         SELECT receipts._receipt_id, DATE_TRUNC('month', date_scanned) as month_scanned FROM receipts
         )
         SELECT month_scanned, COUNT(DISTINCT receipt_items._receipt_id), COUNT(DISTINCT _receipt_item_id) 
         FROM receipt_items
         LEFT JOIN months_scanned ON receipt_items._receipt_id = months_scanned._receipt_id
         GROUP BY month_scanned
         ORDER BY month_scanned DESC
         """).show()

┌─────────────────────────┐
│    max(date_scanned)    │
│        timestamp        │
├─────────────────────────┤
│ 2021-03-01 18:17:34.772 │
└─────────────────────────┘


Unique Receipts Scanned by Month
┌───────────────┬─────────────────────────────┐
│ month_scanned │ count(DISTINCT _receipt_id) │
│     date      │            int64            │
├───────────────┼─────────────────────────────┤
│ 2021-03-01    │                          23 │
│ 2021-02-01    │                         448 │
│ 2021-01-01    │                         640 │
│ 2020-11-01    │                           6 │
│ 2020-10-01    │                           2 │
└───────────────┴─────────────────────────────┘


Unique Receipts and Receipt Items Scanned by Month (Only receipts with receipt_items included)
┌───────────────┬───────────────────────────────────────────┬──────────────────────────────────┐
│ month_scanned │ count(DISTINCT receipt_items._receipt_id) │ count(DISTINCT _receipt_item_id) │
│     date      │         

### ISSUE 2: Most Receipt Items are missing brand codes

In [4]:
print(
    f"Lots of missingness ({round((1-(2600/(2600+4341)))*100, 2)}%) in brand_code information for each rewards receipt item; unreliable"
)
conn.sql(
    """
    SELECT CASE WHEN brand_code IS NOT NULL THEN True ELSE False END as has_brand_code, COUNT(DISTINCT _receipt_item_id) as N_Receipt_Items
    FROM receipt_items 
    GROUP BY has_brand_code
    """
).show()

Lots of missingness (62.54%) in brand_code information for each rewards receipt item; unreliable
┌────────────────┬─────────────────┐
│ has_brand_code │ N_Receipt_Items │
│    boolean     │      int64      │
├────────────────┼─────────────────┤
│ false          │            4341 │
│ true           │            2600 │
└────────────────┴─────────────────┘



### Are we able to merge brands table from barcodes?
+ Yes but, we would only pull in values for 2 items

In [5]:
rewards_barcodes = set(
    conn.sql(
        "SELECT DISTINCT barcode FROM receipt_items WHERE barcode IS NOT NULL"
    ).df()["barcode"]
)
print(f"Unique receipt_items barcodes {len(rewards_barcodes)}")
brands_barcodes = set(
    conn.sql("SELECT DISTINCT barcode FROM brands WHERE barcode IS NOT NULL").df()[
        "barcode"
    ]
)
print(f"Unique brands barcodes {len(brands_barcodes)}")

print(
    f"There are {len(rewards_barcodes - brands_barcodes)} barcodes in rewards items that do not exist in the brands table"
)
conn.sql("""
         WITH brands_barcodes AS (
            SELECT brands.barcode, brand_name, brands.brand_code FROM brands
         )
         SELECT DISTINCT receipt_items.barcode as barcode, receipt_items.brand_code as receipt_brand_code, brands_barcodes.brand_code as brands_brand_code, brands_barcodes.brand_name as brands_brand_name
         FROM receipt_items
         INNER JOIN brands_barcodes ON receipt_items.barcode = brands_barcodes.barcode
         WHERE receipt_brand_code IS NULL
         ORDER BY barcode
         """)

Unique receipt_items barcodes 568
Unique brands barcodes 1160
There are 552 barcodes in rewards items that do not exist in the brands table


┌──────────────┬────────────────────┬───────────────────┬───────────────────┐
│   barcode    │ receipt_brand_code │ brands_brand_code │ brands_brand_name │
│   varchar    │      varchar       │      varchar      │      varchar      │
├──────────────┼────────────────────┼───────────────────┼───────────────────┤
│ 511111101451 │ NULL               │ QUAKER            │ Quaker            │
│ 511111602118 │ NULL               │ JELL-O            │ Jell-O            │
└──────────────┴────────────────────┴───────────────────┴───────────────────┘

### Re-answering What are the top 5 brands by receipts scanned for most recent month?
+ **New assumptions**
    1) Top 5 brands by receipts scanned for February 2021
    2) Compare Top 5 brands against January 2021
    3) Merge brand_codes from brands table using barcodes

##### **The top brands we have codes for are BRAND, MISSION, and VIVA for the month of February 2021**
+ BRAND was in 3 Receipts
+ MISSION was in 2 Receipts
+ VIVA was in 1 Receipt
+ There were 115 receipts that contained items with no associated brand

In [6]:
print("Top 5 Brands by receipts scanned and receipt items scanned")
print("February 2021")
conn.sql("""
    WITH receipt_dates AS (
        SELECT _receipt_id, date_scanned FROM receipts     
    ),
    brand_codes AS (
        SELECT brand_code, barcode FROM brands     
    ),
    ranked_receipt_items AS (
        SELECT
            receipt_items._receipt_id,
            receipt_items._receipt_item_id,
            COALESCE(receipt_items.brand_code, brand_codes.brand_code) as brand_code,
            date_scanned,
            DATE_TRUNC('month', date_scanned)::date as ym,
            DENSE_RANK() OVER (ORDER BY ym DESC) as rank
        FROM receipt_items
        LEFT JOIN receipt_dates ON receipt_items._receipt_id = receipt_dates._receipt_id
        LEFT JOIN brand_codes ON receipt_items.barcode = brand_codes.barcode
        ORDER BY date_scanned DESC
    )
    SELECT 
        brand_code,
        COUNT(DISTINCT _receipt_id) AS N_Receipts,
        COUNT(DISTINCT _receipt_item_id) AS N_Receipt_Items
    FROM ranked_receipt_items
    WHERE rank = 2
    GROUP BY brand_code
    ORDER BY N_Receipts DESC
;""")

Top 5 Brands by receipts scanned and receipt items scanned
February 2021


┌────────────┬────────────┬─────────────────┐
│ brand_code │ N_Receipts │ N_Receipt_Items │
│  varchar   │   int64    │      int64      │
├────────────┼────────────┼─────────────────┤
│ NULL       │        115 │             186 │
│ BRAND      │          3 │               3 │
│ MISSION    │          2 │               2 │
│ VIVA       │          1 │               1 │
└────────────┴────────────┴─────────────────┘

### Re-answering How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
##### **Comparing February 2021 to January 2021, we can see there was a sharp drop in the amount of receipts scanned as well as receipt items scanned going from 6655 -> 192 items from January to February**
##### **While showing ties, the top brands by number of receipts containing a brand code for the month of January 2021 are Ben and Jerrys, Pepsi, Folgers, Kraft, Kellogg's, Kleenex, Borden, and Dole**
+ Ben and Jerrys was in 32 Receipts
+ Pepsi was in 23 Receipts
+ Folgers was in 23 Receipts
+ Kraft was in 22 Receipts
+ Kellogg's was in 22 Receipts
+ Kleenex was in 21 Receipts
+ Borden was in 21 Receipts
+ Dole was in 20 Receipts
+ There were 504 receipts that contained items with no associated brand


In [7]:
print("Top 5 Brands by receipts scanned and receipt items scanned")
print("January 2021")
conn.sql("""
    WITH receipt_dates AS (
        SELECT _receipt_id, date_scanned FROM receipts     
    ),
    brand_codes AS (
        SELECT brand_code, barcode FROM brands     
    ),
    ranked_receipt_items AS (
        SELECT
            receipt_items._receipt_id,
            receipt_items._receipt_item_id,
            COALESCE(receipt_items.brand_code, brand_codes.brand_code) as brand_code,
            date_scanned,
            DATE_TRUNC('month', date_scanned)::date as ym,
            DENSE_RANK() OVER (ORDER BY ym DESC) as rank
        FROM receipt_items
        LEFT JOIN receipt_dates ON receipt_items._receipt_id = receipt_dates._receipt_id
        LEFT JOIN brand_codes ON receipt_items.barcode = brand_codes.barcode
        ORDER BY date_scanned DESC
    ),
    brands_by_receipts AS (
        SELECT 
            brand_code,
            COUNT(DISTINCT _receipt_id) AS N_Receipts,
            COUNT(DISTINCT _receipt_item_id) AS N_Receipt_Items,
            DENSE_RANK() OVER (ORDER BY N_Receipts DESC) as N_Receipts_Rank
        FROM ranked_receipt_items
        WHERE rank = 3
        GROUP BY brand_code
        ORDER BY N_Receipts DESC
    )
    SELECT * FROM brands_by_receipts
    WHERE N_Receipts_Rank <=6

;""")

Top 5 Brands by receipts scanned and receipt items scanned
January 2021


┌────────────────┬────────────┬─────────────────┬─────────────────┐
│   brand_code   │ N_Receipts │ N_Receipt_Items │ N_Receipts_Rank │
│    varchar     │   int64    │      int64      │      int64      │
├────────────────┼────────────┼─────────────────┼─────────────────┤
│ NULL           │        504 │            4059 │               1 │
│ BEN AND JERRYS │         32 │             180 │               2 │
│ PEPSI          │         23 │              93 │               3 │
│ FOLGERS        │         23 │              38 │               3 │
│ KRAFT          │         22 │              60 │               4 │
│ KELLOGG'S      │         22 │              27 │               4 │
│ KLEENEX        │         21 │              88 │               5 │
│ BORDEN         │         20 │              71 │               6 │
│ DOLE           │         20 │              53 │               6 │
└────────────────┴────────────┴─────────────────┴─────────────────┘

In [8]:
conn.close()